In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load dataset
data = pd.read_csv("tsunami_dataset.csv")

In [2]:
# Extract relevant columns
df = data[['LATITUDE', 'LONGITUDE', 'EQ_MAGNITUDE', 'EQ_DEPTH']]

df.head()

,LATITUDE,LONGITUDE,EQ_MAGNITUDE,EQ_DEPTH
0,40.00,25.000,7.0,NaN
1,51.45,-2.583,NaN,NaN
2,36.08,36.250,NaN,NaN
3,40.00,128.000,NaN,NaN
4,39.96,26.240,6.0,NaN


In [3]:
eq_depth_median = df['EQ_DEPTH'].median()
eq_depth_mean = df['EQ_DEPTH'].mean()
eq_depth_mean
eq_depth_median

31.0

In [4]:
# Drop rows with missing target values
df = df.dropna(subset=['EQ_MAGNITUDE']).copy()
# Fill missing values in 'EQ_DEPTH' with the mean depth
mean_depth = df['EQ_DEPTH'].mean()
df['EQ_DEPTH'].fillna(mean_depth, inplace=True)

# Split Data again after filling missing values
X = df[['LATITUDE', 'LONGITUDE']]
y_magnitude = df['EQ_MAGNITUDE']
y_depth = df['EQ_DEPTH']
X_train, X_test, y_magnitude_train, y_magnitude_test, y_depth_train, y_depth_test = train_test_split(X, y_magnitude, y_depth, test_size=0.2, random_state=42)


C:\Users\Sujay S C\AppData\Local\Temp\ipykernel_19968\3180732683.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['EQ_DEPTH'].fillna(mean_depth, inplace=True)


In [5]:
df.head()

,LATITUDE,LONGITUDE,EQ_MAGNITUDE,EQ_DEPTH
0,40.000,25.000,7.0,36.644544
4,39.960,26.240,6.0,36.644544
24,36.404,25.396,6.3,36.644544
33,36.404,25.396,6.2,36.644544
39,38.400,22.300,7.1,36.644544


In [6]:
from sklearn.ensemble import RandomForestRegressor


In [7]:
# Model for Magnitude
model_magnitude = RandomForestRegressor(warm_start=False)
model_magnitude.fit(X_train, y_magnitude_train)

# Model for Depth
model_depth = RandomForestRegressor(warm_start=False)
model_depth.fit(X_train, y_depth_train)


RandomForestRegressor()

In [8]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# For Magnitude
y_magnitude_pred = model_magnitude.predict(X_test)
magnitude_mse = mean_squared_error(y_magnitude_test, y_magnitude_pred)
magnitude_mae = mean_absolute_error(y_magnitude_test, y_magnitude_pred)
print("Magnitude MSE:", magnitude_mse)
print("Magnitude MAE:", magnitude_mae)

# For Depth
y_depth_pred = model_depth.predict(X_test)
depth_mse = mean_squared_error(y_depth_test, y_depth_pred)
depth_mae = mean_absolute_error(y_depth_test, y_depth_pred)
print("Depth MSE:", depth_mse)
print("Depth MAE:", depth_mae)

Magnitude MSE: 0.5955780656563661
Magnitude MAE: 0.5997199192897497
Depth MSE: 1174.77564114924
Depth MAE: 14.41657673129842


In [9]:
# Make predictions for a new data point
new_data_point = [[12.9351,77.5347]]  # Insert the latitude and longitude values of the new data point here

# Predict magnitude
magnitude_prediction = model_magnitude.predict(new_data_point)
print("Predicted Magnitude:", magnitude_prediction[0])

# Predict depth
depth_prediction = model_depth.predict(new_data_point)
print("Predicted Depth:", depth_prediction[0])


Predicted Magnitude: 7.335000000000001
Predicted Depth: 28.269572553430827


c:\Users\Sujay S C\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\Sujay S C\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [10]:
import pickle
with open('earthquake_model_magnitude.pkl','wb') as file:
    pickle.dump(model_magnitude,file)
with open('earthquake_model_depth.pkl','wb') as file:
    pickle.dump(model_depth,file)